This notebook is a copy of /om2/user/ckauf/perturbed-neural-nlp/ressources/stimuli_creation/create-concatenated-control.ipynb


To run it: 
srun -t 03:00:00 --mem=30G --pty bash

cd /om2/user/ckauf/perturbed-neural-nlp

source /om2/user/ckauf/anaconda39/etc/profile.d/conda.sh

conda activate perturbed3.8

cd /om/user/gretatu/perturbed-neural-nlp/

unset XDG_RUNTIME_DIR (alias xdg)

jupyter notebook --ip=0.0.0.0 --port=1111 --no-browser


Here, we create materials based on chatGPT https://chat.openai.com/chat on 20230206.

In [1]:
import re
from pathlib import Path
from os.path import abspath
import os
import numpy as np
import random
import pickle
import csv
import subprocess
import pandas as pd

In [7]:
save = False

# Fetch stimulus set to have a csv we can pass to chatGPT

In [2]:
importpath = abspath('../..')
# importpath = '/om2/user/ckauf/perturbed-neural-nlp'
os.chdir(importpath)
print(os.getcwd())

/om/user/gretatu/perturbed-neural-nlp


In [3]:
from neural_nlp.benchmarks.neural import *
import neural_nlp
from neural_nlp.stimuli import StimulusSet
import xarray as xr

benchmark_pool = [
    # primary benchmarks
    ('Pereira2018-encoding', PereiraEncoding),
]
benchmark_pool = {identifier: LazyLoad(lambda identifier=identifier, ctr=ctr: ctr(identifier=identifier))
                  for identifier, ctr in benchmark_pool}

# fetch stimulus set
benchmark = benchmark_pool['Pereira2018-encoding']
stimuli_df = benchmark._target_assembly.attrs['stimulus_set']
stimuli_df

/om2/user/ckauf/anaconda39/envs/perturbed3.8/lib/python3.8/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


Loading lookup from /om2/user/ckauf/anaconda39/envs/perturbed3.8/lib/python3.8/site-packages/brainio_collection/lookup.csv


/om2/user/ckauf/anaconda39/envs/perturbed3.8/lib/python3.8/site-packages/xarray/core/dataarray.py:219: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  elif isinstance(data, pd.Panel):


,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category
0,Beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
1,It is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
2,"As a passive form of agriculture, it does not ...",2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
3,Beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping
4,Artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping
...,...,...,...,...,...,...,...,...
622,Some windows have multiple panes to increase i...,379,384sentences.379,384sentences,384sentences.building_part,95,Window,building_part
623,A woman is a female human adult.,380,384sentences.380,384sentences,384sentences.human,96,Woman,human
624,A woman is stereotypically seen as a caregiver.,381,384sentences.381,384sentences,384sentences.human,96,Woman,human
625,A woman can become pregnant and bear children.,382,384sentences.382,384sentences,384sentences.human,96,Woman,human


In [7]:
# Save copy of the stimulus df
stimuli_df_copy = stimuli_df.copy(deep=True)
if save:
    stimuli_df_copy.to_csv('Pereira2018_stimulus_set_20220209.csv')

## Prep the stimulus_df

In [8]:
#get original sentences
original_sentences = list(stimuli_df_copy.sentence.values)

In [6]:
# Same regex used in e.g. 2_create_information-loss-manipulation.ipynb

In [7]:
stimuli_df_copy['sentence_stripped'] = [re.sub(r'[^\w\d\s\'\-\$\%]+', '', sent.lower()) + "." for sent in original_sentences]

In [8]:
stimuli_df_copy

,sentence,sentence_num,stimulus_id,experiment,story,passage_index,passage_label,passage_category,sentence_stripped
0,Beekeeping encourages the conservation of loca...,0,243sentences.0,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping,beekeeping encourages the conservation of loca...
1,It is in every beekeeper's interest to conserv...,1,243sentences.1,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping,it is in every beekeeper's interest to conserv...
2,"As a passive form of agriculture, it does not ...",2,243sentences.2,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping,as a passive form of agriculture it does not r...
3,Beekeepers also discourage the use of pesticid...,3,243sentences.3,243sentences,243sentences.beekeeping,1,beekeeping,beekeeping,beekeepers also discourage the use of pesticid...
4,Artisanal beekeepers go to extremes for their ...,4,243sentences.4,243sentences,243sentences.beekeeping,2,beekeeping,beekeeping,artisanal beekeepers go to extremes for their ...
...,...,...,...,...,...,...,...,...,...
622,Some windows have multiple panes to increase i...,379,384sentences.379,384sentences,384sentences.building_part,95,Window,building_part,some windows have multiple panes to increase i...
623,A woman is a female human adult.,380,384sentences.380,384sentences,384sentences.human,96,Woman,human,a woman is a female human adult.
624,A woman is stereotypically seen as a caregiver.,381,384sentences.381,384sentences,384sentences.human,96,Woman,human,a woman is stereotypically seen as a caregiver.
625,A woman can become pregnant and bear children.,382,384sentences.382,384sentences,384sentences.human,96,Woman,human,a woman can become pregnant and bear children.


In [9]:
# Add number of words (separated by whitespace) in each sentence
stimuli_df_copy['num_words_sentence_stripped'] = [len(x.split(' ')) for x in stimuli_df_copy.sentence_stripped] 

In [10]:
stimuli_df_copy.iloc[5].sentence_stripped

'artisanal honey-making emphasizes quality and character over quantity and consistency.'

In [11]:
stimuli_df_copy.iloc[5].num_words_sentence_stripped

10

## Create chatGPT paraphrase benchmark

The prompt was:

For each sentence, generate 3 paraphrases. Index each sentence with 384sentences.{number}.{1,2,3}.

In [16]:
for i, row in enumerate(stimuli_df_copy.itertuples()):
    print(row.stimulus_id + ' ' + row.sentence)
    if i % 8 ==7:
        print('\nFor each sentence, generate 3 paraphrases. Index each sentence with 384sentences.{number}.{1,2,3}.\n') # pbs manullay change between expt nums

243sentences.0 Beekeeping encourages the conservation of local habitats.
243sentences.1 It is in every beekeeper's interest to conserve local plants that produce pollen.
243sentences.2 As a passive form of agriculture, it does not require that native vegetation be cleared to make way for crops.
243sentences.3 Beekeepers also discourage the use of pesticides on crops, because they could kill the honeybees.
243sentences.4 Artisanal beekeepers go to extremes for their craft, but their product is worth the effort.
243sentences.5 Artisanal honey-making emphasizes quality and character over quantity and consistency.
243sentences.6 To produce the finest honey, beekeepers become micromanagers of their honeybees.
243sentences.7 They scout the fields, know when nectar flows, and select the best ways to extract honey.

For each sentence, generate 3 paraphrases. Index each sentence with 384sentences.{number}.{1,2,3}.

243sentences.8 As the beekeeper opens the hive, the deep hum of 40,000 bees fill

In [30]:
for i, item in enumerate(stimuli_df_copy.sentence_stripped.values):
    print(i + 'item)
    if i % 6 == 5:
        print(f'\nChunk {i} \n ')

beekeeping encourages the conservation of local habitats.
it is in every beekeeper's interest to conserve local plants that produce pollen.
as a passive form of agriculture it does not require that native vegetation be cleared to make way for crops.
beekeepers also discourage the use of pesticides on crops because they could kill the honeybees.
artisanal beekeepers go to extremes for their craft but their product is worth the effort.
artisanal honey-making emphasizes quality and character over quantity and consistency.

Chunk 5 
 
to produce the finest honey beekeepers become micromanagers of their honeybees.
they scout the fields know when nectar flows and select the best ways to extract honey.
as the beekeeper opens the hive the deep hum of 40000 bees fills the air.
the beekeeper checks honey stores pollen supplies and the bee nursery.
bees crawl across his bare arms and hands but they don't sting because they're gentle.
i have a recurring dream about exams even though i'm no longer 

These sentences were copied into chatGPT, paraphrases were generated and pasted into the file called: chatGPT_perturbedNLP_paraphrase.xlsx


This excel was converted into a csv and the chatGPT materials were compiled and filtered using: filter_chatGPT_materials.py


This file stored chatGPT_perturbedNLP_paraphrase_compiled_for-manual-selection.csv


Which underwent manual selection and editing. The merged outcome of this was: chatGPT_perturbedNLP_paraphrase_compiled_for-manual-selection_GT_CK_merge.csv


Which was then cleaned up in compile_materials_post_man_selection.py


# Make chatGPT paraphrase into a neuralNLP benchmark

In [4]:
# We want to sub in the paraphrased sentences in col 'sentence_paraphrase_post_inspection_stripped' in the 'sentence' col in the Pereira stimset. 
stimuli_gpt = pd.read_csv(f'standalone_scripts/stimuli_creation/chatGPT_csvs/Pereira2018_stimulus_set_chatGPT.csv')

In [5]:
assert(stimuli_gpt.sentence_original.values == stimuli_df.sentence.values).all()
assert(stimuli_gpt.stimulus_id.values == stimuli_df.stimulus_id.values).all()
assert(len(stimuli_gpt.sentence_paraphrase_post_inspection_stripped.values) == len(stimuli_df.sentence.values))

In [6]:
assert(stimuli_gpt['sentence_paraphrase_post_inspection_stripped'].nunique() == len(stimuli_df))

In [7]:
# Assert that all is lower-case
assert(stimuli_gpt['sentence_paraphrase_post_inspection_stripped'].apply(lambda sent: sent.islower()).all())

In [8]:
# Sub in the paraphrased sentences into the stimuli df "sentence"

In [9]:
stimuli_df_copy = stimuli_df.copy(deep=True)

In [10]:
sentence_paraphrase = stimuli_gpt['sentence_paraphrase_post_inspection_stripped'].tolist()

In [11]:
sentence_paraphrase

['beekeeping plays a role in conserving local ecosystems.',
 'conserving regional vegetation that provides pollen is of the utmost importance for beekeepers.',
 'this form of agriculture which is passive in nature does not necessitate removal of native plants for crop cultivation.',
 'beekeepers are against the use of pesticides on crops as they could harm the honeybees.',
 'artisanal beekeepers go to great lengths for their craft but their product is worth it.',
 'artisanal honey-making values quality and uniqueness over quantity and uniformity.',
 'to achieve the best quality honey beekeepers closely monitor their honeybee populations.',
 'by exploring the lands and observing nectar patterns they are able to collect the honey effectively.',
 'as the beekeeper begins to access the hive the drone of 40000 bees fills the air.',
 'the beekeeper inspects the honey stores the availability of pollen and the bee nursery.',
 "despite crawling on his unprotected arms and hands the bees don't s

In [12]:
stimuli_df_copy['sentence'] = sentence_paraphrase

In [13]:
stimuli_df_copy.sentence.values # Great, they have final periods and apostrophes

array(['beekeeping plays a role in conserving local ecosystems.',
       'conserving regional vegetation that provides pollen is of the utmost importance for beekeepers.',
       'this form of agriculture which is passive in nature does not necessitate removal of native plants for crop cultivation.',
       'beekeepers are against the use of pesticides on crops as they could harm the honeybees.',
       'artisanal beekeepers go to great lengths for their craft but their product is worth it.',
       'artisanal honey-making values quality and uniqueness over quantity and uniformity.',
       'to achieve the best quality honey beekeepers closely monitor their honeybee populations.',
       'by exploring the lands and observing nectar patterns they are able to collect the honey effectively.',
       'as the beekeeper begins to access the hive the drone of 40000 bees fills the air.',
       'the beekeeper inspects the honey stores the availability of pollen and the bee nursery.',
       "d

In [17]:
os.chdir('/om/user/gretatu/perturbed-neural-nlp/standalone_scripts/stimuli_creation/')

In [20]:
# os.mkdir('chatGPT_stimuli_dfs')

In [14]:
save = True

In [18]:
os.getcwd()

'/om/user/gretatu/perturbed-neural-nlp/standalone_scripts/stimuli_creation'

In [19]:
if save:
    fname = f"chatGPT_stimuli_dfs/stimuli_chatGPT.pkl"
    with open(fname, 'wb') as fout:
        pickle.dump(stimuli_df_copy, fout)